In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Oppgave 1 - Forstå hvordan datasettene og transformermodellen er strukturert

#### 1.1

#### 1.3

Cross entropy er gitt ved $L(θ, D) = -\frac{1}{D \cdot n} \sum_{i=0}^{D-1} \sum_{j=0}^{n-1} \log \hat{Y}_{k,j}^{(i)}$ hvor D er datapunktene, $\theta $
er parameterne, og $\hat{Y}$ er sannsynlighetsfordelingen til den predikterte modellen, samt er $j$ og $i$ dimensjonene til $\hat{Y}$. Det objektfunksjonen gjør er å sammenligne onehot(y) med $\hat{Y}$. Hvis $L(θ, D) = 0$ vil den optimerte modellen og onehot(y) være identiske. Når dette inntreffer vil $argmax_{\text{col}}(\hat{Y})$ = $\hat{y}$ som igjen er lik $y$. I dette tilfellet er $y = [4,3,2,1]$, som også vil være lik $\hat{y}$.
$\hat{Y}$ vil være gitt av den diskrete sannsynlighetsfordelingen: $\hat{Y}$ =
$\left[\begin{array}{ccc}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0\\
0 & 1 & 0 & 0\\
1 & 0 & 0 & 0\\
\end{array}\right]$, som er lik onehot([4,3,2,1]). Dette betyr i praksis at paramtetrene i transformenmodellen klarer å prediktere hva som kommer videre i sekvensen og vi ender opp med samme antatt løsning ($\hat{y}$) som faktisk løsning.




#### 1.4

Med gitte variabler for $d, m, n_{max}, k, p$ og  $L$ er antall enkeltparametre mulig å bestemme. Enkeltparametre er gitt med $w \in \mathbb{R}$ noe som uttrykkes med å se på dimensjoner for ulike parametermatriser i transformermodellen.

$W_E$ og $W_P$ har henholdsvis dimensjonene $W_E \in \mathbb{R}^{d \times m}$ og $ W_P \in \mathbb{R}^{d \times n_{max}} $ som representerer parametermatrisen til en sekvens for x med lengde n, som skrives som $z_0$. I tilegg ønskes det å gjøre $L$ paramtriserte trasformasjoner på $z_0$, så man ender opp med $L \cdot (d \times m + d \times n_{max})$ for embedding delen av enkeltparamtrene. Under unenbeddingen oppstår en ny parametermatrise $W_U$ som er en sekvens med lengde $n$ med heltall opp til $m$, den har dimensjonene $ W_U \in \mathbb{R}^{d \times m} $. Attention-lag bidrar også til antall enkeltparamtre for transformmodellen, der har man 4 parametermatriser; $W_O, W_V, W_Q, W_K$ alle med samme dimensjon $\mathbb{R}^{k \times d} $. Transformermodellen har også en $feed$-$forward $ del som bidrar med to paramtermatriser $W_1$ og $W_2$ begge med dimensjoner $\mathbb{R}^{p \times d} $


Hvis man tar disse parametermatrisene i betrakning og antar at $k < d < p$ vil man ha: 
$w = d \times m+L\cdot (d \times m + d \times n_{max}) + 4 \cdot k \times d + 2 \cdot p \times d $, enkeltparametre. (siden k og p er heltall man bestemmer selv er dette en rimelig antagelse å ta).


#### 1.5

$L=n = n_{max}= x = 1$ og $m=d=k=p=d = 2$ og alle parametermatrisene lik 
$\left[\begin{array}{ccc}
1 & 0 \\
0 & 1 
\end{array}\right]$ = $ I_{2\times 2}$ utenom $W_E$ som er $\left[\begin{array}{ccc}
1 & 0 \\
0 & \alpha
\end{array}\right]$ og $W_P$ = $\left[\begin{array}{ccc}
1 \\
0 
\end{array}\right]$

Med dette oppgitt vil  $ X = onehot(x) = \left[\begin{array}{ccc}0 \\1 
\end{array}\right]$ som resulterer i en $z_0 = \left[\begin{array}{ccc}0 \\ \alpha 
\end{array}\right]+ \left[\begin{array}{ccc}1 \\ 0 
\end{array}\right]$ =$\left[\begin{array}{ccc}1 \\ \alpha 
\end{array}\right]$. For å videre bestemme et uttrykk for $\hat{z}$ må vi se på hva transformermodellen gjør med $z_0$. videre er 

$z_{1/2}$ = $z_0 + W_O^T  W_V  z_0 A(z_0)$ hvor $A(z_0)$ = $softmax_{col}(z_0^T W_Q^T W_K z_0+D)$ hvor D sørger for at den strengt nedre delen av A er 0.
Ved å løse $A(z_0)$ får man utrykket $(1+ \alpha ^2)$ i softmax funksjonen.

$z_{1/2} = \left[\begin{array}{ccc}1 \\ \alpha 
\end{array}\right] + I_{2 \times 2} I_{2 \times 2} \left[\begin{array}{ccc}1 \\ \alpha 
\end{array}\right] softmax(1+ \alpha ^2)$ = $ 2 \left[\begin{array}{ccc}1 \\ \alpha 
\end{array}\right]$ fordi softmax av et utrykk blir å dele på seg selv i e-potens som blir 1.

for $z_1$ får vi et uttrykk som er $z_{1/2} + W_2^T \sigma (W_1 z_{1/2})$, $\sigma$ er en aktiveringsfunskjon, i dette tilfelle kan man bruke $relu(W_1 z_{1/2})$.
Utrykket blir da:

$z_1 = 2  \left[\begin{array}{ccc}1 \\ \alpha 
\end{array}\right]+ I_{2 \times 2} max(0,I_{2 \times 2} 2 \left[\begin{array}{ccc}1 \\ \alpha 
\end{array}\right]) $ = $ 4  \left[\begin{array}{ccc}1 \\ \alpha 
\end{array}\right]$
 
Ved hjelp av $z_1$ kan man ta i bruk likning (8) for å finne sannsynlighetsfunksjonen $Z$. $Z = softmax_{col}(W_U^T Z_1)$, argumentet $W_U^T z_1$ blir lik $z_1$ og softmax gir oss $Z = \frac{1}{e^4 + e^{4 \alpha}} \left[\begin{array}{ccc}e^4 \\ e^{4 \alpha }
\end{array}\right]$

for å få $\hat{z} = [1]$ må $argmax(Z)$ bli 1, og dette krever at verdien på indeks [1] må være større enn den på indeks [0], da må $e^4 < e^{4 \alpha}$ og dette impliserer at $\alpha >1$ 


